In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn
import datetime as dt
import sys
import scipy.stats as stats
import urllib
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm
import torch
from torch import cuda




In [ ]:
test = pd.read_csv("C:/vscode/open/test.csv")
train = pd.read_csv("C:/vscode/open/train.csv")

In [6]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))


# 칼럼 관리

In [ ]:
columns = train.columns

for i in columns:
    print("unique {}:{}".format(i, len(train[i].unique())))
    print("count null {}:{}".format(i, train[i].isnull().sum()))
    print("")

TABLE의 칼럼들 중, CANCELLED(항공편 취소됨)과 DIVERTED(경유여부) 칼럼은 NULL도 없으며 모든 관측값들이 동일합니다. 따라서 분석에 영향을 주지 못한다고 판단하여 해당 칼럼을 제거하였습니다.

In [ ]:
ID = train['ID']

train.drop(columns = ['ID', 'Origin_Airport_ID', 'Destination_Airport_ID', 'Cancelled', 'Diverted'], inplace=True)

In [ ]:
train.columns

이들 중, NULL이 다수 포함된 칼럼은 Estimated_Departure_Time, 'Estimated_Arrival_Time', 'Origin_State', 'Destination_State', 'Airline'의 다섯 칼럼입니다. 

# Time칼럼 분단위 변환

In [ ]:
i = 0
while i < len(train['Estimated_Departure_Time']):
    if train.loc[i, 'Estimated_Departure_Time'] > 0:
        train.loc[i, 'edt'] = (train.loc[i, 'Estimated_Departure_Time']//100)*60 + train.loc[i, 'Estimated_Departure_Time']%100
    i += 1

In [ ]:
i = 0
while i < len(train['Estimated_Arrival_Time']):
    if train.loc[i, 'Estimated_Arrival_Time'] > 0:
        train.loc[i, 'eat'] = (train.loc[i, 'Estimated_Arrival_Time']//100)*60 + train.loc[i, 'Estimated_Arrival_Time']%100
    i += 1

이후, 빈칸을 채우는 데 활용할 수 있도록, eat칼럼에 대한 변형 작업을 거칩니다.
eat < edt라면, 시간대가 변하지 않았다는 가정 하에서는 24시를 넘겨 다음날 도착했다고 보는 것이 타당하므로 해당 조건의 값들에 1440(하루를 분으로 표현)을 더해줍니다.

In [ ]:
i = 0
while i <= 999999:
    if train.loc[i, 'eat'] - train.loc[i, 'edt'] < 0:
        train.loc[i, 'eat'] += 1440
    i += 1

# 날짜(월, 일)칼럼 D(eparture)와 A로 별도 분리

추가로, 세분화해서 활용 가능하도록, Month와 Day_of_Month를 도착일자와 출발일자로 분리합니다. 출발 일자는 변경이 없겠지만, 도착일자는 위의 조건과 같은 상황에 따라 변동이 가능합니다.

In [ ]:
train['dmonth'] = train['Month']
train['dday'] = train['Day_of_Month']

In [ ]:
train['amonth'] = train['Month']
train['aday'] = train['Day_of_Month']

이에, 주별로 공항 데이터를 groupby하여 주 결측값이 있는 데이터에 공항을 통해 주값을 메워주도록 하였습니다. OS(Origin_State)와 DS(Destination_State)를 통해 값을 배정하였습니다.

In [ ]:
OS = train.groupby('Origin_State')['Origin_Airport'].unique().reset_index()
DS = train.groupby("Destination_State")['Destination_Airport'].unique().reset_index()

In [ ]:
type(OS['Origin_Airport'])

In [ ]:
i = 0
while i <= 999999:
    j = 0
    if isinstance(train.loc[i, 'Origin_State'], float):
        while j < 52:
            if train.loc[i, 'Origin_Airport'] in list(OS.loc[j, 'Origin_Airport']):
                train.loc[i, 'Origin_State'] = OS.loc[j, 'Origin_State']
            j += 1
    i += 1

In [ ]:
i = 0
while i <= 999999:
    j = 0
    if isinstance(train.loc[i, 'Destination_State'], float):
        while j < 52:
            if train.loc[i, 'Destination_Airport'] in list(DS.loc[j, 'Destination_Airport']):
                train.loc[i, 'Destination_State'] = DS.loc[j, 'Destination_State']
            j += 1
    i += 1

이렇게 하면 아무래도 100만개짜리 파일이니 어딘가엔 반드시 매칭되는 공항 정보와 주 정보가 있을 것이고, null값을 처리할 수 있을 거라 봅니다. 그러면 이제 train의 null 정보를 확인해보도록 하겠습니다.

In [ ]:
train[['Origin_State', 'Destination_State']].isnull().sum()

?

In [ ]:
i = 0
while i <= 999999:
    j = 0
    if isinstance(train.loc[i, 'Destination_State'], float):
        while j < 52:
            if train.loc[i, 'Destination_Airport'] in list(OS.loc[j, 'Origin_Airport']):
                train.loc[i, 'Destination_State'] = DS.loc[j, 'Origin_State']
            j += 1
    i += 1

In [ ]:
train[['Origin_State', 'Destination_State']].isnull().sum()

In [ ]:
train[train['Destination_State'].isnull()]

In [ ]:
train[train['Origin_Airport'] == 'YNG']

In [ ]:
train[train['Destination_Airport'] == 'YNG']

In [ ]:
train.describe()

In [ ]:
plt.hist(train['Distance'], bins=50)

In [ ]:
train[train['Distance']>3000].head(50)

In [ ]:
i = 0
while i < 999999:
    if train.loc[i, 'edt'] > 0 and train.loc[i, 'eat'] > 0 :
        train.loc[i, 'time'] = train.loc[i, 'eat'] - train.loc[i, 'edt']
    i += 1
    

In [ ]:
plt.scatter( train[train['time'].notnull()][['Distance', 'time']]['Distance'], train[train['time'].notnull()][['Distance', 'time']]['time'])

In [ ]:
train[train['time']>1400]

In [ ]:
distance = train.groupby(['Origin_State', 'Destination_State'])['Distance'].mean().reset_index()

In [ ]:
deptimes = train.groupby(["Origin_State", "Destination_State"])['edt'].mean().reset_index()

In [ ]:
atimes = train.groupby(['Origin_State', 'Destination_State'])['eat'].mean().reset_index()

In [ ]:
distance = distance.merge(deptimes, how='left', on = ['Origin_State', 'Destination_State']).merge(atimes, how='left', on=['Origin_State', 'Destination_State'])

In [ ]:
distance['times'] = distance['eat']-distance['edt']

In [ ]:
list((set(distance[distance['Origin_State']=='California'].index)\
      &set(distance[distance['Destination_State']=='Texas'].index)))[0]

In [ ]:
distance

In [ ]:
train

In [ ]:
distance[distance['Destination_State']==train.loc[5, 'Destination_State']]

In [ ]:
i = 0
while i <= 999999:
    if train.loc[i, 'edt'] > 0:
        pass
    else : 
        if train.loc[i, 'eat'] > 0:
            if len(list(set(distance[distance['Origin_State']==train.loc[i, 'Origin_State']].index)&set(distance[distance['Destination_State']==train.loc[i, 'Destination_State']].index))) != 0:
                train.loc[i, 'edt'] = train.loc[i, 'eat'] - distance.loc[list(set(distance[distance['Origin_State']==train.loc[i, 'Origin_State']].index)&set(distance[distance['Destination_State']==train.loc[i, 'Destination_State']].index))[0], 'times']
            else:
                pass
    i += 1

In [ ]:
i = 0
while i <= 999999:
    if train.loc[i, 'eat'] > 0:
        pass
    else : 
        if train.loc[i, 'edt'] > 0:
            if len(list(set(distance[distance['Origin_State']==train.loc[i, 'Origin_State']].index)&set(distance[distance['Destination_State']==train.loc[i, 'Destination_State']].index))) != 0:
                train.loc[i, 'eat'] = train.loc[i, 'edt'] + distance.loc[list(set(distance[distance['Origin_State']==train.loc[i, 'Origin_State']].index)&set(distance[distance\
                ['Destination_State']==train.loc[i, 'Destination_State']].index))[0], 'times']
            else:
                pass
    i += 1

In [ ]:
train.isnull().sum()

In [ ]:
from sklearn.linear_model import LinearRegression

model = LinearRegression(fit_intercept=False)

model.fit(pd.DataFrame(train[train['time'].notnull()][['Distance', 'time']]['time']), pd.DataFrame(train[train['time'].notnull()][['Distance', 'time']]['Distance']))

print(model.coef_)
print(model.intercept_)

In [ ]:
train

In [ ]:
dpm = 5.38393855

In [ ]:
train['time'] = train['Distance']/dpm

In [ ]:
i = 0
while i <= 999999:
    if train.loc[i, 'edt'] < 0:
        train.loc[i, 'edt'] += 1440
        train.loc[i, 'eat'] += 1440
    i += 1

In [ ]:
i = 0
while i <= 999999:
    if train.loc[i, 'edt'] > 0:
        train.loc[i, 'eat'] = round(train.loc[i, 'eat'])
        train.loc[i, 'edt'] = round(train.loc[i, 'edt'])
    i += 1

In [ ]:
train.to_csv("C:/vscode/open/final/training.csv")

In [ ]:
oa = train.groupby('Origin_Airport')['Airline'].unique().reset_index().dropna()

In [ ]:
da = train.groupby('Destination_Airport')['Airline'].unique().reset_index().dropna()

In [ ]:
ta = train.groupby('Tail_Number')['Airline'].unique().reset_index().dropna()

In [ ]:
import random

In [ ]:
oairlineairport = train.groupby('Origin_Airport')['Airline'].unique().reset_index()

dairlineairport = train.groupby('Destination_Airport')['Airline'].unique().reset_index()

In [ ]:
carr_id = train.groupby('Carrier_ID(DOT)')['Airline'].unique().reset_index()

In [ ]:
ccod = train.groupby("Carrier_Code(IATA)")['Airline'].unique().reset_index()

In [ ]:
i = 0
while i <= 999999:
    j = 0
    k = 0
    l = 0
    if isinstance(train.loc[i, 'Airline'], float):
        
        while j < len(oairlineairport['Origin_Airport']):
            if train.loc[i, 'Origin_Airport'] == oairlineairport.loc[j, 'Origin_Airport']:
                
                while k < len(dairlineairport['Destination_Airport']):
                    
                    if train.loc[i, 'Destination_Airport'] == dairlineairport.loc[k, 'Destination_Airport']:
                        
                        while l < len(carr_id['Carrier_ID(DOT)']) :
                            if train.loc[i, 'Carrier_ID(DOT)'] == carr_id.loc[l, 'Carrier_ID(DOT)']:
                                
                                train.loc[i, 'Airline'] = list(set(oairlineairport.loc[j, 'Airline'])&set(dairlineairport.loc[k, 'Airline'])&set(carr_id.loc[l, 'Airline']))[random.randint(0, (len(set(oairlineairport.loc[j, 'Airline'])&set(dairlineairport.loc[k, 'Airline'])&set(carr_id.loc[l, 'Airline']))-1))]
                                break
                                
                            l += 1
                        break
                        
                    k += 1
                break
                
            j += 1
    i += 1

In [ ]:
i = 0
while i <= 999999:
    j = 0
    if isinstance(train.loc[i, 'Airline'], float):
        while j < len(oa['Origin_Airport']):
            k = 0
            if oa.loc[j, 'Origin_Airport'] == train.loc[i, 'Origin_Airport']:
                while k < len(da['Destination_Airport']):
                    l = 0
                    if da.loc[k, 'Destination_Airport'] == train.loc[i, 'Destination_Airport']:
                        
                        while l < len(ta['Tail_Number']):
                            if ta.loc[l, 'Tail_Number'] == train.loc[i, 'Tail_Number']:
                                if len(set(oa.loc[j, 'Airline'])&set(da.loc[k, 'Airline'])&set(ta.loc[l, 'Airline'])) == 0:
                                    train.loc[i, 'Airline'] = list(set(oa.loc[j, 'Airline'])&set(da.loc[k, 'Airline']))[random.randint(0, int(len(list(set(oa.loc[j, 'Airline'])&set(da.loc[k, 'Airline']))))-1)]
                                    break
                                else:
                                    train.loc[i, 'Airline'] = list(set(oa.loc[j, 'Airline'])&set(da.loc[k, 'Airline'])&set(ta.loc[l, 'Airline']))[random.randint(0, int(len(list(set(oa.loc[j, 'Airline'])&set(da.loc[k, 'Airline'])&set(ta.loc[l, 'Airline']))))-1)]
                                    break
                                break
                            l += 1
                        break        
                    k += 1
                break        
            j += 1
    i += 1

In [ ]:
i = 0
while i <= 999999:
    j = 0
    if isinstance(train.loc[i, 'Airline'], float):
        while j < len(oa['Origin_Airport']):
            k = 0
            if oa.loc[j, 'Origin_Airport'] == train.loc[i, 'Origin_Airport']:
                while k < len(da['Destination_Airport']):
                    l = 0
                    if da.loc[k, 'Destination_Airport'] == train.loc[i, 'Destination_Airport']:
                        
                        while l < len(ccod['Carrier_Code(IATA)']):
                            if ccod.loc[l, 'Carrier_Code(IATA)'] == train.loc[i, 'Carrier_Code(IATA)']:
                                if len(set(oa.loc[j, 'Airline'])&set(da.loc[k, 'Airline'])&set(ccod.loc[l, 'Airline'])) == 0:
                                    train.loc[i, 'Airline'] = list(set(oa.loc[j, 'Airline'])&set(da.loc[k, 'Airline']))[random.randint(0, int(len(list(set(oa.loc[j, 'Airline'])&set(da.loc[k, 'Airline']))))-1)]
                                    break
                                else:
                                    train.loc[i, 'Airline'] = list(set(oa.loc[j, 'Airline'])&set(da.loc[k, 'Airline'])&set(ccod.loc[l, 'Airline']))[random.randint(0, int(len(list(set(oa.loc[j, 'Airline'])&set(da.loc[k, 'Airline'])&set(ta.loc[l, 'Airline']))))-1)]
                                    break
                                break
                            l += 1
                        break        
                    k += 1
                break        
            j += 1
    i += 1

In [ ]:
train.isnull().sum()

In [ ]:
i = 0
while i <= 999999:
    if isinstance(train.loc[i, 'Airline'], float):
        j = 0
        k = 0
        while j < 52:
            if train.loc[i, 'Origin_Airport'] == oa.loc[j, 'Origin_Airport']:
                while k < 52:
                    if train.loc[i, 'Destination_Airport'] == da.loc[k, 'Destination_Airport']:
                        train.loc[i, 'Airline'] = list(set(oa.loc[j, 'Airline'])&set(da.loc[k, 'Airline']))[random.randint(0, len(list(set(oa.loc[j, 'Airline'])&set(da.loc[k, 'Airline'])))-1)]
                    k += 1
            j += 1    
        
    i += 1

In [ ]:
train.isnull().sum()

In [ ]:
while train['Airline'].isnull().sum() > 50:
    i = 0
    while i <= 999999:
        j = 0
        if isinstance(train.loc[i, 'Airline'], float):
            while j < len(oa['Origin_Airport']):
                if train.loc[i, 'Origin_Airport'] == oa.loc[j, 'Origin_Airport']:
                    train.loc[i, 'Airline'] = oa.loc[j, 'Airline'][random.randint(0, int(len(oa.loc[j, 'Airline'])-1))]
                j += 1
        i += 1

In [ ]:
train.isnull().sum()

In [ ]:
os = train.groupby("Origin_State")['Airline'].unique().reset_index()

In [ ]:
i = 0
while i <= 999999:
    j = 0
    if isinstance(train.loc[i, 'Airline'], float):
        while j < 52:
            if train.loc[i, 'Origin_State'] == os.loc[j, 'Origin_State']:
                train.loc[i, 'Airline'] = os.loc[j, 'Airline'][random.randint(0, len(os.loc[j, 'Airline'])-1)]
            j += 1
    i += 1

In [ ]:
train.isnull().sum()

In [ ]:
missing_airport = train[train['Destination_State'].isnull()]['Airline']

In [ ]:

train[train['Airline']=='Allegiant Air']['Destination_State'].mode()
    

In [ ]:
train['Destination_State'] = train['Destination_State'].fillna('Florida')

In [ ]:
train.isnull().sum()

In [ ]:
train = train.drop(columns = ['Carrier_Code(IATA)', 'Carrier_ID(DOT)'])

In [ ]:
train.describe()

In [ ]:
train.to_csv("C:/vscode/open/final/training2.csv")

In [ ]:
train = pd.read_csv("C:/vscode/open/final/training2a.csv")

# edt, eat 칼럼 정리, 날짜변환

In [ ]:
i = 0
while i <= 999999:
    if train.loc[i, 'eat'] >= 1440:
        train.loc[i, 'eat'] -= 1440
        train.loc[i, 'aday'] += 1
    i += 1

In [ ]:
train['edt'].describe()

In [ ]:
train['aday'].describe()

In [ ]:
i = 0
while i <= 999999:
    if train.loc[i, 'aday'] >= 32:
        train.loc[i, 'amonth'] += 1
        train.loc[i, 'aday'] = 1
    i += 1

In [ ]:
i = 0
while i <= 999999:
    if train.loc[i, 'amonth'] == 2:
        if train.loc[i, 'aday'] >= 29:
            train.loc[i, 'amonth'] += 1
            train.loc[i, 'aday'] -= 28
    elif train.loc[i, 'amonth'] in [4, 6, 9, 11]:
        if train.loc[i, 'aday'] >= 31:
            train.loc[i, 'amonth'] += 1
            train.loc[i, 'aday'] -= 30
    i += 1

In [ ]:
train.isnull().sum()

# 테이블 컬럼 정리

In [ ]:
train.drop(columns = ['Month', 'Day_of_Month'], inplace=True)

In [ ]:
train.drop(columns = 'Unnamed: 0', inplace=True)

In [ ]:
train.drop(columns = ['Estimated_Departure_Time', 'Estimated_Arrival_Time'], inplace=True)

In [ ]:
train = train.drop(index = 1000000)

In [ ]:
train

In [ ]:
train = train[['edt', 'eat', 'dmonth', 'dday', 'amonth', 'aday',
       'Origin_Airport', 'Origin_State', 'Destination_Airport',
       'Destination_State', 'Distance', 'Airline', 'Tail_Number', 'time', 'Delay']]

In [ ]:
train.isnull().sum()

# 모델 적합

In [ ]:
train.to_csv("C:/vscode/open/final/training3a.csv")

In [ ]:
train = pd.read_csv("C:/vscode/open/final/training3a.csv")

In [ ]:
data = train[train['Delay'].notnull()]

In [ ]:
target = data['Delay']
data.drop(columns = 'Delay', inplace=True)

In [ ]:
data

In [ ]:
target = target.replace('Not_Delayed', 0).replace('Delayed', 1)

In [ ]:
data = data.reset_index()

In [ ]:
data

In [ ]:
target=target.reset_index()

In [ ]:
i = 0
while i < len(data['edt']):
    data.loc[i, 'edt'] = float(data.loc[i, 'edt']//60)*100 + float((data.loc[i, 'edt']%60)/60)
    data.loc[i, 'eat'] = float(data.loc[i, 'eat']//60)*100 + float((data.loc[i, 'eat']%60)/60)
    i += 1

In [ ]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()

In [ ]:
train.to_csv("C:/vscode/open/final/training3b.csv")

In [ ]:
encoder1 = encoder.fit(data['Origin_Airport'])
data["Origin_Airport"] = encoder1.transform(data['Origin_Airport'])
data['Destination_Airport'] = encoder1.transform(data['Destination_Airport'])

In [ ]:
encoder2 = encoder.fit(data['Destination_State'])
data['Origin_State'] = encoder2.transform(data['Origin_State'])
data['Destination_State'] = encoder2.transform(data['Destination_State'])

In [ ]:
encoder3 = encoder.fit(data['Airline'])
data['Airline'] = encoder3.transform(data['Airline'])

In [ ]:
data2 = pd.read_csv("C:/vscode/open/final/dta.csv")

In [ ]:
encoder4 = encoder.fit(data2['Tail_Number'])
data2['Tail_Number'] = encoder4.transform(data2['Tail_Number'])

In [ ]:
encoder4 = encoder.fit(data['Tail_Number'])
data['Tail_Number'] = encoder4.transform(data['Tail_Number'])

In [ ]:
data = data.drop(columns = 'index')

In [ ]:
data = data.drop(columns = 'Unnamed: 0')

In [ ]:
data.dtypes

In [ ]:
if 'Unnamed: 0.1' in data.columns:
    data.drop(columns = 'Unnamed: 0.1', inplace=True)

In [ ]:
data['Origin_Airport']=data['Origin_Airport'].astype('category')

In [ ]:
data['Destination_Airport']=data['Destination_Airport'].astype('category')

In [ ]:
data['Origin_State'] = data['Origin_State'].astype('category')

In [ ]:
data['Destination_State'] = data['Destination_State'].astype('category')

In [ ]:
data['Airline'] = data['Airline'].astype('category')

In [ ]:
data['Tail_Number'] = data['Tail_Number'].astype('category')

In [ ]:
col = ['Origin_Airport', 'Destination_Airport', 'Origin_State', 'Destination_State', 'Airline', 'Tail_Number']

In [ ]:
for i in col:
    data[i] = data[i].cat.as_unordered()

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
scaler = StandardScaler()

In [ ]:
scaler.fit(pd.DataFrame(data['Distance']))

In [ ]:
data.dtypes

In [ ]:
target

In [ ]:
target.drop(columns = 'index', inplace=True)

In [ ]:
target['Delay'] = target['Delay'].astype('int')

In [ ]:
i = 0
while i < len(target['Delay']):
    if target.loc[i, 'Delay'] == 0:
        target.loc[i, 'Delayed'] = 0
        target.loc[i, 'Not_Delayed'] = 1
    else:
        target.loc[i, 'Delayed'] = 1
        target.loc[i, 'Not_Delayed'] = 0
    i += 1

In [ ]:
target.drop(columns = 'Delay', inplace=True)

target['Delayed'] = target['Delayed'].astype('int')
target['Not_Delayed'] = target['Not_Delayed'].astype('int')

In [ ]:
from sklearn.model_selection import train_test_split

xtrain, xtest, ytrain, ytest = train_test_split(data, target, test_size=0.25)

In [ ]:
import xgboost as xgb

In [ ]:
from xgboost import XGBClassifier

In [ ]:
dtrain = xgb.DMatrix(data = xtrain, label = ytrain, enable_categorical=True)

dtest = xgb.DMatrix(data = xtest, label = ytest, enable_categorical=True)

params = {'max_depth':14, 
          'learning_rate':0.03,  
          'scale_pos_weight':5, 
          'min_child_weight':3, 
          'objective' : 'binary:logistic',
          'eval_metric':'logloss',
          'booster':'gbtree'}

num_rounds = 150

In [ ]:
xgbmodel = xgb.train(params = params, num_boost_round = num_rounds, dtrain=dtrain)

In [ ]:
pred = xgbmodel.predict(dtest)

In [ ]:
from sklearn.metrics import log_loss

In [ ]:
log_loss(ytest, pred)

In [ ]:
pred

In [ ]:
testdata = pd.read_csv("C:/vscode/open/final/testdata4.csv")

In [ ]:
testdata2 = pd.read_csv("C:/vscode/open/final/testdata4.csv")

In [ ]:
for label in np.unique(testdata['Origin_Airport']):
    if label not in encoder1.classes_:
        encoder1.classes_ = np.append(encoder1.classes_, label)

In [ ]:
testdata['Origin_Airport'] = encoder1.transform(testdata['Origin_Airport'])
testdata['Destination_Airport'] = encoder1.transform(testdata['Destination_Airport'])

In [ ]:
testdata['Origin_State'].mode()

In [ ]:
testdata['Destination_State'].mode()

In [ ]:
testdata['Origin_State'] = testdata['Origin_State'].fillna('California')

In [ ]:
testdata['Destination_State'] = testdata['Destination_State'].fillna('California')

In [ ]:
testdata['Origin_State'] = encoder2.transform(testdata['Origin_State'])
testdata['Destination_State'] = encoder2.transform(testdata['Destination_State'])

In [ ]:
testdata['Airline'].mode()

In [ ]:
testdata['Airline'] = testdata['Airline'].fillna("Southwest Airlines Co.")

In [ ]:
for label in np.unique(testdata['Airline']):
    if label not in encoder3.classes_:
        encoder3.classes = np.append(encoder3.classes_, label)

In [ ]:
testdata['Airline'] = encoder3.transform(testdata['Airline'])

In [ ]:
for label in np.unique(testdata['Tail_Number']):
    if label not in encoder4.classes_:
        encoder4.classes_ = np.append(encoder4.classes_, label)

In [ ]:
testdata['Tail_Number'] = encoder4.transform(testdata['Tail_Number'])

In [ ]:
samsub = pd.read_csv("C:/vscode/open/sample_submission.csv")

In [ ]:
testdata.isnull().sum()

In [ ]:
testdata['Distance'] = scaler.transform(pd.DataFrame(testdata['Distance']))

In [ ]:
testdata

In [ ]:
data

In [ ]:
testdata.drop(columns = ['Unnamed: 0'], inplace=True)

In [ ]:
clist = ['Origin_Airport', 'Origin_State', 'Destination_State', 'Destination_Airport', 'Airline', 'Tail_Number']

In [ ]:
for i in clist:
    testdata[i] = testdata[i].astype('category')
    testdata[i] = testdata[i].cat.as_unordered()

In [ ]:
testdata

In [ ]:
testdata.dtypes

# Model Predict I

In [ ]:
samid = samsub['ID']

In [ ]:
samsub.drop(columns = 'ID', inplace=True)

In [ ]:
samsub = samsub[['Delayed', 'Not_Delayed']]

In [ ]:
dsubmit = xgb.DMatrix(data = testdata, label = samsub, enable_categorical=True)

In [ ]:
submis1 = xgbmodel.predict(dsubmit)

In [ ]:
submis1 = pd.DataFrame(submis1)

In [ ]:
samsub1 = pd.concat([samid, submis1], axis=1)

In [ ]:
samsub1.columns = ['ID', 'Delayed', 'Not_Delayed']

In [ ]:
samsub1 = samsub1[["ID", "Not_Delayed", 'Delayed']]

In [ ]:
samsub1

In [ ]:
samsub1 = samsub1.set_index("ID")

In [ ]:
samsub1

In [ ]:
samsub1.to_csv("C:/vscode/open/submission/samsub9.csv")

# Model Predict II

In [ ]:
samsub2 = samsub1.copy()

In [ ]:
samsub2 = samsub2.reset_index()

In [ ]:
i = 0
while i < 1000000:
    samsub2.loc[i, 'Not_Delayed'] = samsub2.loc[i, 'Not_Delayed']/(samsub2.loc[i, 'Not_Delayed']+samsub2.loc[i, "Delayed"])
    samsub2.loc[i, "Delayed"] = samsub2.loc[i, 'Delayed']/(samsub2.loc[i, 'Not_Delayed']+samsub2.loc[i, "Delayed"])
    i += 1

In [ ]:
samsub2 = samsub2.set_index("ID")

In [ ]:
samsub2.to_csv("C:/vscode/open/submission/samsub10.csv")

In [ ]:
samsub3 = samsub2.copy().reset_index()

In [ ]:
i = 0
while i <= 999999:
    samsub3.loc[i, 'Not_Delayed'] = samsub3.loc[i, 'Not_Delayed'] - 0.1
    samsub3.loc[i, 'Delayed'] = samsub3.loc[i, 'Delayed'] +0.1
    i += 1

In [ ]:
samsub3=pd.concat([samid, samsub3], axis=1)

In [ ]:
samsub3 = samsub3.set_index("ID")

In [ ]:
samsub3